# Using Neptune in R

## Before you start

### Install dependencies

In [1]:
install.packages('neptune', dependencies = TRUE)

also installing the dependencies ‘Matrix’, ‘reticulate’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


### Import libraries

In [10]:
library(neptune)

### Initialize Neptune

In [11]:
init_neptune(project_name = 'shared/r-integration',
             api_token = 'ANONYMOUS',
            )

Project(shared/r-integration)

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet.  

---

If you already have a [Neptune account](https://neptune.ai/register), you can create your own experiment and start logging to it using your personal API token. Pass your `username` to the `project_qualified_name` argument of the `neptune.init()` method: `project_qualified_name='YOUR_USERNAME/YOUR_PROJECT_NAME`. If you don't have a project yet, keep `/sandbox` at the end. The `sandbox` project is automatically created for you.

## Quickstart

### Step 1: Create an Experiment

This creates an experiment in Neptune.

Once you have a live experiment you can log things to it. 

In [12]:
create_experiment(name='minimal example')

Experiment(RIN-181)

Click on the link above to open this experiment in Neptune.

For now it is mostly empty but keep the tab with experiment open to see what happens next. 

### Step 2: Log metrics

Log your performance metrics during or after training with the ``log_metric`` function.

In [13]:
log_metric('accuracy', 0.92)

for (i in 0:100){
  log_metric('random_training_metric', i * 0.6)
}

## Step 3: Explore results in the Neptune UI

Go to the experiment link and explore charts that were created from the metric values you logged. 

### Step 4: Stop logging

When you are finished logging you should stop your current Neptune experiment.

In [14]:
stop_experiment()

## Example: Full model training and versioning on a Sonar dataset

Let's do something more real-life:

* train a model on a Sonar dataset
* keep track of model training artifacts and training metrics 

### Install packages and import the dataset

In [15]:
install.packages(c('digest', 'mlbench', 'randomForest'), dependencies = TRUE)

library(digest)
library(mlbench)
library(randomForest)

# load dataset
data(Sonar)
dataset <- Sonar
x <- dataset[,1:60]   # predictors
y <- dataset[,61]     # labels

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.


### Log hyperparameters

In [16]:
params = list(ntree=625,
              mtry=13,
              maxnodes=50
              )

create_experiment(name='training on Sonar',
                  params = params
)

Experiment(RIN-182)

### Log data versions and other properties

In [17]:
set_property(property = 'data-version', value = digest(dataset))

SEED=1234
set.seed(SEED)
set_property(property = 'seed', value = SEED)

### Train model 

In [18]:
model <- randomForest(x = x, y = y,
  ntree=params$ntree, mtry = params$mtry, maxnodes = params$maxnodes,
  importance = TRUE
  )

### Log evaluation metrics

In [19]:
log_metric('mean OOB error', mean(model$err.rate[,1]))
log_metric('error class M', model$confusion[1,3])
log_metric('error class R', model$confusion[2,3])

for (err in (model$err.rate[,1])) {
  log_metric('OOB errors', err)
}

### Log model weights

In [20]:
save(model, file="model.Rdata")
log_artifact('model.Rdata')

### Log performance charts

In [21]:
for (t in c(1,2)){
  jpeg('importance_plot.jpeg')
  varImpPlot(model,type=t)
  dev.off()
  log_image('feature_importance', 'importance_plot.jpeg')
}

### Stop experiment

In [22]:
stop_experiment()

### Explore results in the Neptune UI

Go to the experiment link and explore charts that were created from the metric values you logged. 

You can also check out [this experiment dashboard](https://ui.neptune.ai/o/shared/org/r-integration/experiments?viewId=fa3b57a5-77fb-4edb-83fc-505014d3649d):

![image](https://neptune.ai/wp-content/uploads/r-integration-tour.gif)